In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install python-gdcm
#!pip install pylibjpeg-libjpeg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2,random
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image, ImageStat
import albumentations as A

In [ ]:
os.mkdir("image512")
#os.mkdir("image600_addeqhist")
os.mkdir("image600")
#os.mkdir("image600_both")

In [ ]:
ids = ["0b858129adb4_image",
       "3d12cb6aad8b_image",
       "41e9a794b342_image",
       "681ed0b5dff2_image",
       "0eb641cb0dcd_image",
       "0f9709784c19_image",
       "3b982073ec16_image",
       "4f3d52d652dd_image"]

In [ ]:
df = pd.read_csv("../input/step2-make-dataframe/train_study.csv")
print(len(df),len(ids))
df = df[~df.id.isin(ids)]
print(len(df))
df = df[~df.duplicated(keep='last',subset=['mean', 'var'])]
print(len(df))
df = df.sort_values("norm_var",ascending=False)
df['fold'] = 0
df.fold = df.fold.apply(lambda x: random.randint(0,3))
#df.to_csv("train.csv")
df

In [ ]:
def get_transform(ori_size):
    transform = A.Compose([
        #A.Rotate(p=1.0,limit=5),
        A.RandomSizedCrop(
                         min_max_height=[int(0.8*ori_size), int(0.95*ori_size)],
                         height=ori_size,
                         width=ori_size, 
                         p=1.0)
    ],p=1.0)
    return transform

def exclude_blackflame(image, ori_size, trytimes=10):
    best_var = image.std()
    transform = get_transform(ori_size)
    for _ in range(trytimes):
        sample = transform(image=image)
        if sample['image'].std() < best_var:
            best_var = sample['image'].std()
            images = sample['image']
    if best_var >= image.std():
        images = image
    return images

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def resize(array, size1,size2, keep_ratio=True, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size1, size2), resample)
        im = expand2square(im, background_color=0)
    else:
        im = im.resize((size1, size2), resample)
    
    return im

In [ ]:
dfs = []
dfs.append(df[df.fold ==0])
dfs.append(df[df.fold ==1])
dfs.append(df[df.fold ==2])
dfs.append(df[df.fold ==3])
print(len(dfs[0]),len(dfs[1]),len(dfs[2]),len(dfs[3]))
print(len(df))

In [ ]:
def run(fold):
    for idx,(index,row) in enumerate(dfs[fold].iterrows()):
        image = read_xray(row.path)
        #image = cv2.resize(image,(2048,2048),interpolation = cv2.INTER_AREA)
        image = exclude_blackflame(image, ori_size=2048, trytimes=10)
        save_img = cv2.resize(image,(512,512),interpolation = cv2.INTER_AREA)
        cv2.imwrite("image512" + "/" + row.id + ".png",save_img)
        
        save_img = cv2.resize(image,(600,600),interpolation = cv2.INTER_AREA)
        cv2.imwrite("image600" + "/" + row.id + ".png",save_img)
        
        #plt.imshow(images)
        #plt.show()
        
        #save_img = cv2.resize(image,(600,600),interpolation = cv2.INTER_AREA)
        #cv2.imwrite("image600_both" + "/" + row.id + ".png",save_img)
        #plt.imshow(images)
        #plt.show()
        #cv2.imwrite("image512" + "/" + row.id + ".png",images)
        #images = exclude_blackflame(image, ori_size=1024, trytimes=10)
        #plt.imshow(images)
        #plt.show()
        #cv2.imwrite("image600" + "/" + row.id + ".png",images)
        #images = exclude_blackflame(image, ori_size=1024, trytimes=10)
        #plt.imshow(images)
        #plt.show()
        #cv2.imwrite("image640" + "/" + row.id + ".png",images)
        #break
        #display(im)
        #stat = ImageStat.Stat(im)
        #df.loc[index,'mean'] = stat.mean
        #df.loc[index,'var'] = stat.stddev
        #print(np.array(im).shape)
        #image = cv2.resize(image,(512,512),interpolation = cv2.INTER_AREA)
        #im.save("image" + "/" + row.id_x + ".png")
        if idx%500==0:
            print(idx)

In [ ]:
#run(1)

In [ ]:
from joblib import Parallel, delayed
Parallel(n_jobs=4, backend="threading")(delayed(run)(i) for i in range(4))

In [ ]:
!zip -qr image600.zip image600
!rm -r image600_normal
!zip -qr image512.zip image512
!rm -r image512
#!zip -qr image600_excludeblackframe.zip image600_excludeblackframe
#!rm -r image600_excludeblackframe
#!zip -qr image600_keep.zip image600_keep
#!rm -r image600_keep
#!zip -qr image600_both.zip image600_both
#!rm -r image600_both
#!zip -qr image640_keep.zip image640_keep
#!rm -r image640_keep

In [ ]:
df.to_csv("train.csv")